In [5]:
%pip install synthcity

  Using cached synthcity-0.2.10-py3-none-any.whl (428 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 89.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:0000:0100:01
  Using cached nflows-0.14.tar.gz (45 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.2 MB/s eta 0:00:0000:0100:01
  Using cached lifelines-0.27.7-py3-none-any.whl (409 kB)
  Using cached opacus-1.4.1-py3-none-any.whl (226 kB)
  Using cached decaf_synthetic_data-0.1.6-py3-none-any.whl (9.1 kB)
  Using cached optuna-3.6.1-py3-none-any.whl (380 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 42.7 MB/s eta 0:00:00
  Using cached loguru-0.7.2-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.4 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 4.2 MB/s eta 0:00:0000:0100:01
  Using cached geomloss-0.2.6.tar.gz (

In [87]:
%pip install catenets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 44.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [150]:
from synthcity.plugins import Plugins
from synthcity.utils.serialization import save_to_file, load_from_file
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import pandas as pd
from catenets.models.jax import *
import numpy as np
from sklearn.metrics import mean_squared_error

In [54]:
def train_models(gen_list, datasets, n_runs):
    for g in gen_list:
        for dataset in datasets:
            for i in range(n_runs):
                print(f'Training model {g} iteration {i} on dataset {dataset}')
                model = Plugins().get(g)
                model.fit(datasets[dataset])
                fp = 'models/' + g + '_' + dataset + '_' + str(i) + '.pkl'
                print(f'Saving model {g} iteration {i} on dataset {dataset}')
                save_to_file(fp, model)

In [157]:
def run_cate_experiment(X, y, w, X_t, mu0, mu1, learner):
    learner.fit(X, y, w)
    pred = learner.predict(X_t)
    cate = mu1 - mu0
    rmse = mean_squared_error(cate, pred, squared=False)


    return rmse

In [3]:
gen_list = Plugins(categories=["generic"]).list()

[2024-04-12T03:14:46.478527+0000][34347][CRITICAL] module disabled: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py


In [32]:
datasets = {}

In [123]:
ihdp_full = pd.read_csv('../Datasets/ihdp.csv')

In [124]:
ihdp = ihdp_full.drop(['y_cfactual', 'mu0', 'mu1'], axis=1)

In [5]:
jobs =pd.read_csv('../Datasets/jobs_small.csv')

In [20]:
twins = pd.read_csv('../Datasets/twins.csv')

In [42]:
datasets['jobs'] = jobs
datasets['ihdp'] = ihdp

In [29]:
twins_filled = twins.fillna(0)

In [169]:
train_ihdp = ihdp.loc[[i for i in range(500)]]
test_ihdp = ihdp.loc[[i+500 for i in range(247)]]

In [176]:
t = TNet()
X = np.array(train_ihdp.drop(['treatment', 'y_factual'], axis=1))
y = np.array(train_ihdp['y_factual'])
w = np.array(train_ihdp['treatment'])
X_t = np.array(test_ihdp.drop(['treatment', 'y_factual'], axis=1))
mu0 = ihdp_full.loc[[i+500 for i in range(247)]]['mu0']
mu1 = ihdp_full.loc[[i+500 for i in range(247)]]['mu1']


In [177]:
run_cate_experiment(X, y, w, X_t, mu0, mu1, t)

0.7719368953886587

In [178]:
s = SNet()
run_cate_experiment(X, y, w, X_t, mu0, mu1, s)

1.4724982786068312

In [179]:
r = RNet()
run_cate_experiment(X, y, w, X_t, mu0, mu1, r)

0.9456332934151883

In [180]:
ra = RANet()
run_cate_experiment(X, y, w, X_t, mu0, mu1, ra)

0.684305803764733

In [181]:
s1 = SNet1()
run_cate_experiment(X, y, w, X_t, mu0, mu1, s1)

0.8152373195923046

In [183]:
s2 = SNet2()
run_cate_experiment(X, y, w, X_t, mu0, mu1, s2)

0.812445943128992

In [184]:
s3 = SNet3()
run_cate_experiment(X, y, w, X_t, mu0, mu1, s3)

1.581308492059898

In [185]:
p = PWNet()
run_cate_experiment(X, y, w, X_t, mu0, mu1, p)

3.0835652361522934

In [186]:
x = XNet()
run_cate_experiment(X, y, w, X_t, mu0, mu1, x)

0.8689671530694568

In [196]:
gens = ['ctgan_ihdp_0.pkl', 'ctgan_ihdp_1.pkl','ctgan_ihdp_2.pkl','tvae_ihdp_0.pkl','tvae_ihdp_1.pkl', 'tvae_ihdp_2.pkl', 'nflow_ihdp_0.pkl', 'nflow_ihdp_2.pkl', 'nflow_ihdp_2.pkl', 'ddpm_ihdp_0.pkl', 'ddpm_ihdp_1.pkl', 'ddpm_ihdp_2.pkl']


In [197]:
learners = [TNet(), SNet(), RANet(), PWNet(), DRNet()]
results = pd.DataFrame(columns=['generator', 'learner', 'rmse'])
for fp in gens:
    for learner in learners:
        gen = load_from_file('models/'+fp)
        s = gen.generate(count=500).dataframe()
        X = np.array(s.drop(['treatment', 'y_factual'], axis=1))
        y = np.array(s['y_factual'])
        w = np.array(s['treatment'])
        X_t = np.array(test_ihdp.drop(['treatment', 'y_factual'], axis=1))
        mu0 = ihdp_full.loc[[i+500 for i in range(247)]]['mu0']
        mu1 = ihdp_full.loc[[i+500 for i in range(247)]]['mu1']
        
        rmse = run_cate_experiment(X,  y, w, X_t, mu0, mu1, learner)

        results.loc[len(results.index)] = [fp, learner.__str__(), rmse]

        print(f'rmse for {fp} on {learner} is {rmse}')

rmse for ctgan_ihdp_0.pkl on TNet() is 1.7557168598822808
rmse for ctgan_ihdp_0.pkl on SNet() is 1.8419106698975425
rmse for ctgan_ihdp_0.pkl on RANet() is 1.3595632310760057
rmse for ctgan_ihdp_0.pkl on PWNet() is 3.1984878670186467
rmse for ctgan_ihdp_0.pkl on DRNet() is 2.040555598188145
rmse for ctgan_ihdp_1.pkl on TNet() is 1.622096884211218
rmse for ctgan_ihdp_1.pkl on SNet() is 4.240035380716217
rmse for ctgan_ihdp_1.pkl on RANet() is 1.671540604115676
rmse for ctgan_ihdp_1.pkl on PWNet() is 2.2357591857683734
rmse for ctgan_ihdp_1.pkl on DRNet() is 2.3763259665564966
rmse for ctgan_ihdp_2.pkl on TNet() is 1.3571055233610043
rmse for ctgan_ihdp_2.pkl on SNet() is 1.9812444476211613
rmse for ctgan_ihdp_2.pkl on RANet() is 2.1165015051746545
rmse for ctgan_ihdp_2.pkl on PWNet() is 3.161964997061986
rmse for ctgan_ihdp_2.pkl on DRNet() is 1.6196267660458314
rmse for tvae_ihdp_0.pkl on TNet() is 2.4496204835156146
rmse for tvae_ihdp_0.pkl on SNet() is 4.200910385651328
rmse for tvae

In [201]:
results['type'] = [i[:4] for i in results['generator']]

['ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'ctga',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'tvae',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'nflo',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm',
 'ddpm']

In [206]:
learners = [TNet(), SNet(), RANet(), PWNet(), DRNet()]


for learner in learners:


    X = np.array(train_ihdp.drop(['treatment', 'y_factual'], axis=1))
    y = np.array(train_ihdp['y_factual'])
    w = np.array(train_ihdp['treatment'])
    X_t = np.array(test_ihdp.drop(['treatment', 'y_factual'], axis=1))
    mu0 = ihdp_full.loc[[i+500 for i in range(247)]]['mu0']
    mu1 = ihdp_full.loc[[i+500 for i in range(247)]]['mu1']
    
    rmse = run_cate_experiment(X,  y, w, X_t, mu0, mu1, learner)

    results.loc[len(results.index)] = ['real', learner.__str__(), rmse, 'real']

    print(f'rmse for real on {learner} is {rmse}')

rmse for real on TNet() is 0.7719368953886587
rmse for real on SNet() is 1.4724982786068312
rmse for real on RANet() is 0.684305803764733
rmse for real on PWNet() is 3.0835652361522934
rmse for real on DRNet() is 0.7561913303944524


In [209]:
results.groupby(['type', 'learner']).mean()

rmse
type learner          
ctga DRNet()  2.012169
     PWNet()  2.865404
     RANet()  1.715868
     SNet()   2.687730
     TNet()   1.578306
ddpm DRNet()  1.559844
     PWNet()  3.756681
     RANet()  1.871510
     SNet()   2.354893
     TNet()   2.004037
nflo DRNet()  3.620621
     PWNet()  4.466600
     RANet()  3.451242
     SNet()   4.299772
     TNet()   3.341471
real DRNet()  0.756191
     PWNet()  3.083565
     RANet()  0.684306
     SNet()   1.472498
     TNet()   0.771937
tvae DRNet()  3.288469
     PWNet()  3.508094
     RANet()  2.888534
     SNet()   3.923766
     TNet()   2.559388